In [24]:
from google.colab import files
uploaded = files.upload()

Saving lab_events_alive_matched_part1.csv to lab_events_alive_matched_part1.csv
Saving lab_events_alive_matched_part2.csv to lab_events_alive_matched_part2.csv
Saving lab_events_alive_matched_part3.csv to lab_events_alive_matched_part3.csv
Saving lab_events_dead_matched.csv to lab_events_dead_matched.csv


In [25]:
import math
import numpy as np
import pandas as pd

data1 = pd.read_csv("lab_events_alive_matched_part1.csv")
data2 = pd.read_csv("lab_events_alive_matched_part2.csv")
data3 = pd.read_csv("lab_events_alive_matched_part3.csv")
data4 = pd.read_csv("lab_events_dead_matched.csv")
frames = [data1, data2, data3, data4];
data = pd.concat(frames, ignore_index = True)
print(data)

         subject_id  itemid         charttime  valuenum flag
0               406   51484  21/03/2120 15:45       NaN  NaN
1               406   51486  21/03/2120 15:45       NaN  NaN
2               406   51487  21/03/2120 15:45       NaN  NaN
3               406   51491  21/03/2120 15:45      5.00  NaN
4               406   51492  21/03/2120 15:45     30.00  NaN
...             ...     ...               ...       ...  ...
3318575        9611   51274           41:00.0     16.10    1
3318576        9611   51275           41:00.0     43.40    1
3318577        9611   51277           41:00.0     18.60    1
3318578        9611   51279           41:00.0      2.61    1
3318579        9611   51301           41:00.0      8.50    0

[3318580 rows x 5 columns]


In [26]:
# Lab events
# Performed min/max normalization on valuenum for each corresponding itemid 
# (specifically on ALIVE AND DEAD ICU patients who met inclusion/exclusion criteria)
# Formula = (valuenum - min)/(max - min)

item_ids = data['itemid'].unique()   #find unique itemids
len(item_ids)                        #find length of itemids
size = (len(item_ids),3)             #define dimension of new matrix
min_max_matrix  = np.zeros(size)     #create matrix that will store corresponding min and max of valuenum for every itemid

for x in range(len(item_ids)):      
  min_max_matrix[x,0] = item_ids[x]
  min_max_matrix[x,1] = data['valuenum'].loc[(data['itemid'] == item_ids[x]) & (data['valuenum'] != 'NaN')].min()
  min_max_matrix[x,2] = data['valuenum'].loc[(data['itemid'] == item_ids[x]) & (data['valuenum'] != 'NaN')].max()

for y in range(len(data)):
    result = np.where(min_max_matrix == data['itemid'].loc[y]);
    if np.isnan(min_max_matrix[result[0][0]][1]):
      min_max_matrix[result[0][0]][1] = 100001;
    elif np.isnan(min_max_matrix[result[0][0]][2]):
      min_max_matrix[result[0][0]][2] = 100001;
    elif min_max_matrix[result[0][0]][1] == min_max_matrix[result[0][0]][2]:
      min_max_matrix[result[0][0]][1] = 100001;
      min_max_matrix[result[0][0]][2] = 100001;
    elif (min_max_matrix[result[0][0]][1] != 100001) & (min_max_matrix[result[0][0]][2] != 100001):
      data['valuenum'].loc[y] = (data['valuenum'].loc[y] -  min_max_matrix[result[0][0]][1])/(min_max_matrix[result[0][0]][2] - min_max_matrix[result[0][0]][1]);

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#print(data.loc[1:30]) #ranges from 0 to 3,318,580
#let us break down the data frame to 4 different dataframe of lengths 10^6 (0-999,999), 10^6 (1,000,000 - 1,999,999), 10^6 (2,000,000 - 2,999,999), and 318,581 (3,000,000 - 3,318,580)
data_frame1 = data.loc[0:999999];

data_frame2 = data.loc[1000000:1999999];

data_frame3 = data.loc[2000000:2999999];

data_frame4 = data.loc[3000000:3318580];



In [36]:
data_frame1.to_csv(r"labevents_minmax_normalized_part1.csv", index = False)
data_frame2.to_csv(r"labevents_minmax_normalized_part2.csv", index = False)
data_frame3.to_csv(r"labevents_minmax_normalized_part3.csv", index = False)
data_frame4.to_csv(r"labevents_minmax_normalized_part4.csv", index = False)
